<a href="https://colab.research.google.com/github/hkdan502/NLP_Basics/blob/main/qa_train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치하기

TPU 관련 패키지를 설치합니다. TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요. GPU를 쓴다면 아래 라인을 실행할 필요가 없습니다.

In [1]:
# TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요.
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

TPU 이외에 의존성 있는 패키지를 설치합니다.

In [2]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.0 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=7002032f4da86ed728a9be1753e98dba658a7f0c6c7dc8809ddf2f825c8be116
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c1

# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [4]:
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
args = QATrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="korquad-v1", #파인튜닝할 다운스트림 태스크 데이터
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-qa",
    max_seq_length=128, #입력 문장 최대 길이(질문/지문 모두 포함)
    max_query_length=32, #질문 최대 길이
    doc_stride=64, #지문에서 몇 개 토큰을 *슬라이딩하면서 데이터를 늘릴지 결정
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

# 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정합니다.

In [5]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


# 로거 설정
메세지 출력 등을 위한 logger를 설정합니다.

In [6]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=2, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, fp16=False, tpu_cores=0, tqdm_enabled=True)
INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=2, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=

# 말뭉치 다운로드
실습에 사용할 말뭉치(KorQuAD-V1)를 다운로드합니다.

In [7]:
nlpbook.download_downstream_dataset(args)

Downloading: 38.5MB [00:01, 30.6MB/s]
Downloading: 3.88MB [00:00, 27.3MB/s]


# 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다

In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

# 학습데이터 구축

### KorQuADV1Corpus 클래스가 하는 일
- json 포맷의 KorQuAD 데이터를 QAExample로 읽어오고, QADataset이 요구하면 QAExample을 QADataset에 제공한다. 

# KorQuADV1Corpus가 넘겨준 0번 QAExample에 대한 QADataset클래스의 중간 산출 과정
1. QADataset 클래스는 QAExample의 question_text를 질문으로, context_text를 지문으로 취급하고 'CLS 질문 SEP 지문 SEP'으로 이어붙인 뒤 토큰화를 수행한다. 
2. 입력 문장(question+context)의 최대 길이인 max_seq_length와 질문 최대 길이인 max_query_length를 각각 128, 32로 설정해 두었으므로 전체 토큰 개수와 질문 토큰 개수가 이보다 많아지지 않도록 토큰화한다.
3. QAExample의 answer_text와 start_position_character를 활용해 정답 시작/끝 토큰을 찾는다. 

In [9]:
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
corpus = KorQuADV1Corpus()

#QADataset 클래스는 KorQuADV1Corpus로부터 QAExample을 받아, 토크나이저로 QAFeatures로 변환한다.
train_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from train dataset file at /content/Korpora/korquad-v1
INFO:ratsnlp:Creating features from train dataset file at /content/Korpora/korquad-v1
convert squad examples to features: 100%|██████████| 57688/57688 [06:10<00:00, 155.71it/s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:question & context: [CLS] 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트를 읽고 무엇을 쓰고 ##자 했 ##는가 ? [SEP] 18 ##3 ##9년 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트 ##을 처음 읽고 그 내용 ##에 마음이 끌려 이를 소재 ##로 해서 하나의 교 ##향 ##곡 ##을 쓰 ##려는 뜻을 갖 ##는다 . 이 시기 바 ##그 ##너 ##는 18 ##3 ##8년 ##에 빛 독 ##촉 ##으로 산 ##전 ##수 ##전을 다 걲 ##은 상황이 ##라 좌 ##절 ##과 실망 ##에 가득 ##했 ##으며 메 ##피 ##스 ##토 ##펠 ##레스 ##를 만나는 파 ##우스 ##트 ##의 심 ##경 ##에 공감 ##했다고 한다 . 또한 파리 ##에서 아 ##브 ##네 ##크 ##의 지휘 ##로 파리 음악 ##원 관 ##현 ##악 ##단이 연 ##주 ##하는 베 ##토 [SEP]
INFO:ratsnlp:question & context: [CLS] 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트를 읽고 무엇을 쓰고 ##자 했 ##는가 ? [SEP] 18 ##3 ##9년 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트 ##을 처음 읽고 그 내용 ##에 마음이 끌려 이를 소재 ##로 해서 

### train_dataset[0]
- input_ids : 원본 문장을 토큰화한 뒤 인덱싱을 수행한 결과
- attention_mask : 해당 토큰이 패딩(0)인지 실제 데이터(1)인지 나타냄. train_dataset[0]은 패딩 토큰이 전혀 없으므로 모두 1
- token_type_ids : 세그먼트 정보로, 'CLS question SEP'에 해당하면 0, 'context SEP'에 해당하면 1, 나머지 패딩인 경우 0을 준다. 질문과 지문의 토큰 수는 각각 17, 108개 이므로 0으로 채우는 첫번째 세그먼트 길이는 CLS,SEP을 합쳐 19개, 두번째 세그먼트 길이는 SEP을 합쳐 109개가 된다. 나머지는 세그먼트는 0으로 모두 채워지는데 max_seq_length인 128에서 첫 세그먼트 길이인 19개, 두 번째 세그먼트 길이인 109개를 뺀 나머지인 10개가 0으로 채워진다.
- start_positions, end_positions : 각각 input_ids에서 정답 토큰의 시작과 끝 인덱스를 의미한다. train_dataset[0]의 정답인 '교향곡'의 input_ids에서 인덱스는 45, 47이 시작과 끝 인덱스이다. 

In [16]:
train_dataset[0]

QAFeatures(input_ids=[2, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 19143, 13985, 12449, 9194, 4105, 3385, 9411, 32, 3, 8601, 4633, 29697, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 4104, 4027, 8793, 13985, 391, 9132, 4113, 10966, 11728, 12023, 14657, 4091, 8598, 16639, 341, 4573, 4771, 4027, 2139, 8478, 14416, 214, 8202, 17, 2451, 13007, 1480, 4313, 4538, 4008, 8601, 4633, 22903, 4113, 1676, 868, 4913, 7965, 1789, 4203, 4110, 15031, 786, 250, 4057, 10878, 4007, 2593, 4094, 4128, 10289, 4113, 10958, 4062, 9511, 1355, 4600, 4103, 4775, 5602, 10770, 4180, 26732, 3231, 23243, 4104, 4042, 2015, 4012, 4113, 9198, 8763, 8129, 17, 10384, 23008, 7971, 2170, 4408, 4011, 4147, 4042, 17015, 4091, 23008, 21056, 4165, 323, 4175, 4158, 11413, 2273, 4043, 7966, 1543, 4775, 3], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

# 테스트 데이터 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [10]:
val_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


INFO:ratsnlp:Creating features from val dataset file at /content/Korpora/korquad-v1
INFO:ratsnlp:Creating features from val dataset file at /content/Korpora/korquad-v1
convert squad examples to features: 100%|██████████| 5533/5533 [00:36<00:00, 151.77it/s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:question & context: [CLS] 198 ##9년 6월 30 ##일 평양 ##축 ##전에 대표로 파견 된 인물 ##은 ? [SEP] 198 ##9년 2월 15일 여의도 농민 폭력 시위를 주도 ##한 혐의 ( 폭력 ##행위 ##등 ##처벌 ##에 ##관 ##한 ##법 ##률 ##위반 ) 으로 지명 ##수 ##배 ##되었다 . 198 ##9년 3월 12 ##일 서울 ##지 ##방 ##검찰 ##청 공안 ##부는 임종석 ##의 사전 ##구속 ##영장 ##을 발부 ##받았 ##다 . 같은 해 6월 30 ##일 평양 ##축 ##전에 임 ##수 ##경을 대표로 파견 ##하여 국가보안법 ##위반 혐의가 추가 ##되었다 . 경찰은 12월 18 ##일 ~ 20 ##일 사이 서울 경 ##희 ##대학 ##교 ##에서 임종석 ##이 성 ##명 발표 ##를 추진 ##하고 있다는 첩 ##보를 입 ##수 ##했고 , 12월 18 ##일 오전 7시 40 ##분 경 가스 ##총 ##과 [SEP]
INFO:ratsnlp:question & context: [CLS] 198 ##9년 6월 30 ##일 평양 ##축 ##전에 대표로 파견 된 인물 ##은 ? [SEP] 198 ##9년 2월 15일 여의도 농민 폭력 시위를 주도 ##한 혐의 ( 폭력 ##행위 ##등 ##처벌 ##에 ##관 ##한 ##법 ##률 ##

# 모델 초기화
프리트레인이 완료된 BERT 모델을 읽고, 질의 응답을 수행할 모델을 초기화합니다.

In [11]:
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)

In [12]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model c

# 학습 준비
Task와 Trainer를 준비합니다.

In [13]:
from ratsnlp.nlpbook.qa import QATask
task = QATask(model, args)

In [14]:
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


# 학습
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-qa`)에 저장됩니다.

In [15]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-qa exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                     | Params
---------------------------------------------------
0 | model | BertForQuestionAnswering | 108 M 
---------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.318   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
